# Input Functions

#### import Modules 

In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from textblob import TextBlob
import re
import csv

## CHANGE PATH FOR YOU MACHINE

In [ ]:
# Define the paths to the files
gift_card_path = 'raw_data/amazon_reviews_us_Gift_Card_v1_00.tsv'
major_appliances_path = 'raw_data/amazon_reviews_us_Major_Appliances_v1_00.tsv'
shoes_path = 'raw_data/amazon_reviews_us_Shoes_v1_00.tsv'
electronics_path =  'raw_data/amazon_reviews_us_Electronics_v1_00.tsv'

# Read the files into dataframes
df_gift_card = pd.read_csv(gift_card_path, sep='\t', error_bad_lines=False)
df_major_appliances = pd.read_csv(major_appliances_path, sep='\t', error_bad_lines=False)
df_shoes = pd.read_csv(shoes_path, sep='\t', error_bad_lines=False)
df_electronics = pd.read_csv(electronics_path, sep='\t', error_bad_lines=False)


# Display the first few rows of each dataframe to verify
print("Gift Card Reviews DataFrame:")
display(df_gift_card.head(1))
print("\nMajor Appliances Reviews DataFrame:")
display(df_major_appliances.head(1))
print("Shoes Reviews DataFrame:")
display(df_shoes.head(1))
print("Electronics Reviews DataFrame:")
display(df_electronics.head(1))

# Read Data & Filter Based On User Inputs

In [5]:
import pandas as pd

# Define the file paths for the datasets
gift_card_path = 'raw_data/amazon_reviews_us_Gift_Card_v1_00.tsv'
major_appliances_path = 'raw_data/amazon_reviews_us_Major_Appliances_v1_00.tsv'
shoes_path = 'raw_data/amazon_reviews_us_Shoes_v1_00.tsv'
electronics_path = 'raw_data/amazon_reviews_us_Electronics_v1_00.tsv'

def read_dataset(dataset_path):
    return pd.read_csv(dataset_path, sep='\t', error_bad_lines=False, warn_bad_lines=False)

def merge_datasets(selected_category):
    paths = {
        'Gift Card': gift_card_path,
        'Major Appliances': major_appliances_path,
        'Shoes': shoes_path,
        'Electronics': electronics_path
    }
    if selected_category in paths:
        return read_dataset(paths[selected_category])
    elif selected_category == 'all':
        dfs = [read_dataset(path) for path in paths.values()]
        return pd.concat(dfs, ignore_index=True)

def remove_specific_columns(df):
    df.drop(columns=["customer_id", "review_id", "product_id"], inplace=True)
    return df

def modify_review_date_to_year(df):
    df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce').dt.year
    df.dropna(subset=['review_date'], inplace=True)
    df['review_date'] = df['review_date'].astype(int)
    return df

def categorize_votes(df, column_names):
    for column in column_names:
        category_col_name = f'{column}_category'
        df[category_col_name] = 'No Votes'  # Default category for 0 votes
        has_votes = df[column] > 0
        votes_data = df.loc[has_votes, column]
        
        unique_values = votes_data.unique()
        if len(unique_values) > 4:
            try:
                df.loc[has_votes, category_col_name] = pd.qcut(votes_data, q=4, labels=["Minimal Engagement", "Low Engagement", "Moderate Engagement", "High Engagement"])
            except ValueError:
                df.loc[has_votes, category_col_name] = pd.cut(votes_data, bins=4, labels=["Minimal Engagement", "Low Engagement", "Moderate Engagement", "High Engagement"])
        else:
            df.loc[has_votes, category_col_name] = pd.cut(votes_data, bins=4, labels=["Minimal Engagement", "Low Engagement", "Moderate Engagement", "High Engagement"])
    return df

def get_user_category_selection():
    print("Please select a product category: 'Gift Card', 'Major Appliances','Shoes', 'Electronics' or 'all'")
    category = input("Your choice: ").strip()
    if category not in ['Gift Card', 'Major Appliances', 'Shoes','Electronics', 'all']:
        print("Invalid selection. Defaulting to 'all'.")
        category = 'all'
    elif category == '':
        category = 'all'
    return category

def get_user_variable_selections():
    print("Available variables: marketplace, product_title, product_category, star_rating, vine, verified_purchase, review_headline, review_body, review_date, helpful_votes_category, total_votes_category")
    selections = input("Enter the variables you're interested in, separated by commas (e.g., product_category, star_rating): ").strip()
    if selections == '':
        return []
    return [var.strip() for var in selections.split(',')]

def display_and_select_unique_values(df, selected_variables):
    filters = {}
    for variable in selected_variables:
        if variable in df.columns:
            unique_values = df[variable].dropna().unique()
            print(f"Unique values for {variable}: {', '.join(unique_values.astype(str))}")
            selected_values = input(f"Enter the values you're interested in for {variable}, separated by commas (leave blank to skip): ")
            if selected_values:
                filters[variable] = [value.strip() for value in selected_values.split(',')]
    return filters

def filter_dataset(df, filters):
    for variable, selected_values in filters.items():
        if variable in df.columns:
            df = df[df[variable].isin(selected_values)]
    return df

if __name__ == "__main__":
    selected_category = get_user_category_selection()
    df = merge_datasets(selected_category)
    print(f"Dataset for '{selected_category}' category loaded.")
    
    df = remove_specific_columns(df)
    print("Removed specific columns: customer_id, review_id, product_id")
    
    df = modify_review_date_to_year(df)
    print("Modified 'review_date' to retain the year only.")
    
    df = categorize_votes(df, ['helpful_votes', 'total_votes'])
    print("Categorized 'helpful_votes' and 'total_votes' into engagement levels.")

    selected_variables = get_user_variable_selections()
    if selected_variables:
        filters = display_and_select_unique_values(df, selected_variables)
        print(f"Selected filters: {filters}")

        filtered_df = filter_dataset(df, filters)
        print("Filtered dataset based on your selections:")
        print(filtered_df.head(1))
    else:
        print("No variable selections made. Displaying first entries of the dataset.")
        print(df.head(1))


Please select a product category: 'Gift Card', 'Major Appliances','Shoes', 'Electronics' or 'all'
Your choice: Major Appliances


/var/folders/3h/r5l1t4656pbc6bzdv94m9wrm0000gn/T/ipykernel_97676/2463294523.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  return pd.read_csv(dataset_path, sep='\t', error_bad_lines=False, warn_bad_lines=False)
/var/folders/3h/r5l1t4656pbc6bzdv94m9wrm0000gn/T/ipykernel_97676/2463294523.py:10: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  return pd.read_csv(dataset_path, sep='\t', error_bad_lines=False, warn_bad_lines=False)


Dataset for 'Major Appliances' category loaded.
Removed specific columns: customer_id, review_id, product_id
Modified 'review_date' to retain the year only.
Categorized 'helpful_votes' and 'total_votes' into engagement levels.
Available variables: marketplace, product_title, product_category, star_rating, vine, verified_purchase, review_headline, review_body, review_date, helpful_votes_category, total_votes_category
Enter the variables you're interested in, separated by commas (e.g., product_category, star_rating): helpful_votes_category
Unique values for helpful_votes_category: No Votes, Minimal Engagement, Low Engagement, Moderate Engagement, High Engagement
Enter the values you're interested in for helpful_votes_category, separated by commas (leave blank to skip): Moderate Engagement, High Engagement
Selected filters: {'helpful_votes_category': ['Moderate Engagement', 'High Engagement']}
Filtered dataset based on your selections:
      marketplace  product_parent  \
66150          U